In [1]:
import pandas
import torch
from nltk.tokenize import word_tokenize, sent_tokenize
from torch import nn
import re

In [111]:
def filter_dataset(dataset):
    # I may have filtered too hard
    # I removed everything between braces because that's just LateX I tried using regex but couldn't get it to work
    # Then I removed empty lines and lines that don't start with an upper case (removing this results in a bunch of random letters in the dataset)
    filtered_dataset = dataset
    print(f'The dataset is size {len(filtered_dataset)} without filtering')
    # with open('latex.txt', 'w') as w: # use this to test wether it removes too much or too little
    #     w.write(''.join(re.findall(r' {6}\n {8}.*?(?:\\displaystyle|\\textstyle).*?\n', filtered_dataset, flags=re.DOTALL)))
    filtered_dataset = re.sub(r' {8}.*?(?:\\displaystyle|\\textstyle).*?\n', '', filtered_dataset, flags=re.DOTALL) # We lowercase the data and remove the LateX
    print(f'The dataset is size {len(filtered_dataset)} without the LaTeX')
    print(f"There are currently {len(re.findall(r'displaystyle', filtered_dataset))} LaTeX blocks that have to be manually deleted")
    # filtered_dataset = '\n'.join([line for line in filtered_dataset.splitlines() if line.strip()]) #  and line[0].isupper() and len(line) > 30
    # print(len(filter_dataset))
    filtered_dataset = re.sub(r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{2,256}\.[a-z]{2,4}\b([-a-zA-Z0-9@:%_\+.~#?&//=]*)', '', filtered_dataset)
    print(f'The dataset is size {len(filtered_dataset)} without the links')
    # I found the regex above here https://regexr.com/37i6s
    return filtered_dataset

In [67]:
def load_dataset(name: str, force_filter=False):
    from pathlib import Path
    if Path('filtered_' + name).exists() and not force_filter:
        return open('filtered_'+name, 'r').read()
    wikis = open(name, 'r').read().split('__WIKI__')
    filtered_dataset = filter_dataset()
    with open('filtered_'+dataset_name, 'w') as o:
        o.write(filtered_dataset)
    return filtered_dataset

In [112]:
dataset_name = 'dataset.txt' # CHOOSING WIKIPEDIA WAS A MISTAKE t
dataset = load_dataset(dataset_name,force_filter=True)

The dataset is size 19739132 without filtering
The dataset is size 15207373 without the LaTeX
There are currently 0 LaTeX blocks that weren't deleted
The dataset is size 15202688 without the links


In [83]:
class Skipgram(nn.Module):
    def __init__(self, vocab_size, embedding_dim, max_norm=1):
        super(Skipgram, self).__init__()
        self.embeddings = nn.Embedding(
            num_embeddings=vocab_size,
            embedding_dim=embedding_dim,
            max_norm=max_norm
        )
        self.linear = nn.Linear(in_features=embedding_dim, out_features=vocab_size)
    def forward(self, inputs):
        pass

In [118]:
sents = sent_tokenize(dataset)
words = [word_tokenize(sent) for sent in sents]
words[:1]

[['machine',
  'learning',
  '(',
  'ml',
  ')',
  'is',
  'a',
  'field',
  'of',
  'study',
  'in',
  'artificial',
  'intelligence',
  'concerned',
  'with',
  'the',
  'development',
  'and',
  'study',
  'of',
  'statistical',
  'algorithms',
  'that',
  'can',
  'learn',
  'from',
  'data',
  'and',
  'generalize',
  'to',
  'unseen',
  'data',
  ',',
  'and',
  'thus',
  'perform',
  'tasks',
  'without',
  'explicit',
  'instructions',
  '.']]